In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix

In [2]:
# --- 1. SIMULATE DATA ---
# N_SAMPLES: Number of samples (patients)
# N_GENES: Number of genes (features)
N_SAMPLES = 1000
N_GENES = 500

# Feature matrix (Expression data)
X = np.random.rand(N_SAMPLES, N_GENES).astype(np.float32) * 10
# Binary Labels (e.g., Disease vs. Healthy)
y = np.random.randint(0, 2, N_SAMPLES)
# A list of gene names for later biomarker identification
gene_names = [f'Gene_{i+1}' for i in range(N_GENES)]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 2. RESHAPE DATA FOR CNNs ---

# 1D CNN Input Shape: (N_samples, N_genes/sequence_length, 1/channels)
X_train_1d = X_train[..., np.newaxis]
X_test_1d = X_test[..., np.newaxis]
print(f"1D CNN Input Shape: {X_train_1d.shape}")

# 2D CNN Input Shape: (N_samples, H, W, Channels)
# We treat the N_GENES (500) as a 2D matrix (e.g., 25x20)
H = 25
W = 20
assert H * W == N_GENES, "H*W must equal N_GENES (500)"

X_train_2d = X_train.reshape(-1, H, W, 1)
X_test_2d = X_test.reshape(-1, H, W, 1)
print(f"2D CNN Input Shape: {X_train_2d.shape}")

NameError: name 'np' is not defined

In [ ]:
# --- 3. 1D CNN MODEL ---
def build_1d_cnn(input_shape):
    model = keras.Sequential([
        layers.Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=input_shape),
        layers.MaxPool1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid') # Binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# --- 4. VANILLA 2D CNN MODEL ---
def build_2d_cnn(input_shape):
    model = keras.Sequential([
        layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPool2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid') # Binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# --- 5. Define Callbacks ---
# Monitor validation loss and stop if it doesn't improve for 5 epochs
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Instantiate and Train the 1D CNN
model_1d = build_1d_cnn(input_shape=(N_GENES, 1))
print("\n--- Training 1D CNN with Early Stopping ---")

# Train the model, passing the callback
history = model_1d.fit(
    X_train_1d, y_train,
    epochs=100,  # Set a high number, as Early Stopping will stop it
    batch_size=32,
    validation_data=(X_test_1d, y_test),
    callbacks=[early_stopping],
    verbose=0
)

# You can check how many epochs ran
print(f"Training finished after {len(history.history['loss'])} epochs.")

# --- 6. Calculate Predictions ---
# Get prediction probabilities for the test set
y_pred_proba = model_1d.predict(X_test_1d).flatten()
y_pred_class = (y_pred_proba > 0.5).astype(int)

# --- Compute Confusion Matrix ---
# The confusion matrix is structured as:
# [[TN, FP],
#  [FN, TP]]
cm = confusion_matrix(y_test, y_pred_class)
print("\n--- Confusion Matrix ---")
print(cm)

# Extract values from the matrix
TN, FP, FN, TP = cm.ravel()

# --- Calculate Metrics ---
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print("\n--- Classification Performance Metrics ---")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity:          {specificity:.4f}")
print(f"Accuracy:             {(TP + TN) / (TP + TN + FP + FN):.4f}")

In [ ]:
# --- 7. GUIDED GRADIENT SALIENCY VISUALIZATION (BIOMARKER EXTRACTION) ---
def guided_gradient_saliency(model, X_input):
    """
    Computes Guided Gradient Saliency for a Keras model.
    This method computes the gradient of the output (prediction)
    with respect to the input features (gene expression).
    """
    # 1. Select the correct output tensor for the target class (index 0 for binary output)
    target_output = model.output[0]

    # 2. Compute the gradient of the output with respect to the input
    # (using the 1D CNN's first layer input tensor)
    input_tensor = model.input
    
    # Keras/TF implementation of Guided Backpropagation (Guided Gradient)
    # The standard Keras gradient does not automatically implement the 
    # Guided ReLU. We use a simple Gradient Saliency/Class Activation
    # map approach as the closest native Keras method.
    # For true Guided Gradient, a custom Keras graph modification is required.
    
    # Using standard Gradient Saliency (Input Gradient)
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(input_tensor)
        # Get the prediction for the target class (e.g., class 1)
        prediction = model(input_tensor)
        
    # Calculate gradient of the prediction w.r.t the input
    saliency_map = tape.gradient(prediction, input_tensor)

    # Convert the saliency map to a numpy array for visualization
    # We take the absolute value as both positive and negative gradients
    # indicate feature importance.
    saliency_scores = np.mean(np.abs(saliency_map.numpy()), axis=0).flatten()

    return saliency_scores

# Get the saliency scores for all test samples
saliency_scores = guided_gradient_saliency(model_1d, X_test_1d)

# Map scores back to gene names
saliency_df = pd.DataFrame({
    'Gene': gene_names,
    'Saliency_Score': saliency_scores
})

# Identify top biomarker genes
N_BIOMARKERS = 20
biomarker_genes_df = saliency_df.sort_values(by='Saliency_Score', ascending=False).head(N_BIOMARKERS)

print(f"\n--- Top {N_BIOMARKERS} Biomarker Genes (Extracted via Saliency) ---")
print(biomarker_genes_df)

# --- 6. VISUALIZATION ---

plt.figure(figsize=(12, 6))
plt.bar(biomarker_genes_df['Gene'], biomarker_genes_df['Saliency_Score'], color='skyblue')
plt.xticks(rotation=45, ha='right')
plt.title(f'Top {N_BIOMARKERS} Genes by CNN Saliency Score')
plt.ylabel('Feature Importance (Absolute Gradient Score)')
plt.xlabel('Gene')
plt.tight_layout()
plt.show()

# --- 7. 2D CNN Training (Optional) ---
# model_2d = build_2d_cnn(input_shape=(H, W, 1))
# print("\n--- Training 2D CNN ---")
# model_2d.fit(X_train_2d, y_train, epochs=10, batch_size=32, validation_data=(X_test_2d, y_test), verbose=0)
# loss, acc = model_2d.evaluate(X_test_2d, y_test, verbose=0)
# print(f"2D CNN Test Accuracy: {acc*100:.2f}%")